# Resnet Supervised Learning
This file trains a supervised learning model (Resnet50) using transfer learning to train on an unknown dataset. And the results give us a baseline accuracy to compare with the semi-supervised model (Mixmatch).

In [ ]:
from keras.utils import np_utils
import numpy as np
from resnet_lib import create_dataset, create_model, test_callback, tune_model, visualize_model

In [ ]:
# Global constants
NUM_CLASSES = 2
IMAGE_SIZE = 64
BATCH_SIZE = 64
EPOCHS = 10

# Directories
INPUT_TF_TRAIN = '../Mixmatch/ML_DATA/SSL/streetview_v3_64.1@250-label.tfrecord'
INPUT_TF_TEST = '../Mixmatch/ML_DATA/streetview_v3_64-test.tfrecord'

In [ ]:
# Construct the datsets
X_train, y_train = create_dataset(INPUT_TF_TRAIN)
X_test, y_test = create_dataset(INPUT_TF_TEST)
y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

In [ ]:
# Build model and fit the top layers
model = create_model(IMAGE_SIZE, NUM_CLASSES, lr=0.0001)
model.fit(x=X_train, y=y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_split = 0.1, validation_freq=1, callbacks=[test_callback((X_test, y_test))])

In [ ]:
# Visualize the model and determine which block will be trained in the tuning phase
visualize_model(model)

In [ ]:
# Fine tune the model
EPOCHS = 30
model = tune_model(model, trainable_layer=143, lr=0.0001)
model.fit(x=X_train, y=y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_split = 0.1, validation_freq=1, callbacks=[test_callback((X_test, y_test))])

In [ ]:
# Output the predictions of a model to csv files for further analysis
pred = model.predict(X_test.astype(np.float32))
np.savetxt("results.csv", np.concatenate((y_test, pred), axis=1), delimiter=",")